# Transfer Entropy

### Hemispheric 

In [6]:
for channel, data in zip(eeg_channels, embedding_data_list_2D):
    print(channel, data.shape)


Fp1 (4227688, 2)
Fpz (4227689, 2)
Fp2 (4227689, 2)
F7 (4227689, 2)
F3 (4227689, 2)
Fz (4227689, 2)
F4 (4227688, 2)
F8 (4227691, 2)
FC5 (4227689, 2)
FC1 (4227689, 2)
FC2 (4227689, 2)
FC6 (4227688, 2)
M1 (4227689, 2)
T7 (4227689, 2)
C3 (4227689, 2)
Cz (4227689, 2)
C4 (4227688, 2)
T8 (4227695, 2)
M2 (4227689, 2)
CP5 (4227689, 2)
CP1 (4227745, 2)
CP2 (4227689, 2)
CP6 (4227725, 2)
P7 (4227689, 2)
P3 (4227689, 2)
Pz (4227689, 2)
P4 (4227689, 2)
P8 (4227689, 2)
POz (4227689, 2)
O1 (4227689, 2)
Oz (4227689, 2)
O2 (4227689, 2)


In [ ]:
import zipfile
import numpy as np
import os
import shutil
from pyinform import transfer_entropy

# Path to the zipped 2D embedded data
zip_file_path = '/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Phase Space/2dembedded_data.zip'
extraction_dir = '/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Phase Space/temp_extraction'

# Extract zipped data to a temporary directory
with zipfile.ZipFile(zip_file_path, 'r') as zipf:
    zipf.extractall(extraction_dir)

eeg_channels = ['Fp1', 'Fpz', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6',
                'M1', 'T7', 'C3', 'Cz', 'C4', 'T8', 'M2', 'CP5', 'CP1', 'CP2', 'CP6',
                'P7', 'P3', 'Pz', 'P4', 'P8', 'POz', 'O1', 'Oz', 'O2']

# Set the desired length
desired_length = 4227688

# Trim each array in embedding_data_list_2D to desired_length
embedding_data_list_2D_trimmed = [data[:desired_length] for data in embedding_data_list_2D]

left_channels = ['Fp1', 'F7', 'F3', 'FC5', 'M1', 'T7', 'C3', 'CP5', 'P7', 'P3', 'O1']
right_channels = ['Fp2', 'F8', 'F4', 'FC6', 'M2', 'T8', 'C4', 'CP6', 'P8', 'P4', 'O2']
central_channels = ['Fpz', 'Fz', 'FC1', 'FC2', 'Cz', 'CP1', 'CP2', 'Pz', 'POz', 'Oz']

# Cleanup: Optionally remove the extraction directory after loading the data
# shutil.rmtree(extraction_dir) 

# Group channels by hemisphere
left_indices = [eeg_channels.index(ch) for ch in left_channels]
right_indices = [eeg_channels.index(ch) for ch in right_channels]

# Average the embedded data for each hemisphere
left_embedding_avg = np.mean([embedding_data_list_2D[i] for i in left_indices], axis=0)
right_embedding_avg = np.mean([embedding_data_list_2D[i] for i in right_indices], axis=0)

# Use one of the dimensions (e.g., x-coordinate) for Transfer Entropy calculation
left_data = left_embedding_avg[:, 0]
right_data = right_embedding_avg[:, 0]

# Compute Transfer Entropy
k, l, m = 1, 1, 1
TE_left_to_right = transfer_entropy(left_data, right_data, k, l, m)
TE_right_to_left = transfer_entropy(right_data, left_data, k, l, m)

print(f"Transfer Entropy from Left to Right (2D embeddings): {TE_left_to_right}")
print(f"Transfer Entropy from Right to Left (2D embeddings): {TE_right_to_left}")


### Regional

### Full Granularity

In [ ]:
def phase_randomize(ts):
    ts_randomized = np.empty_like(ts)
    for i in range(ts.shape[0]):
        # Fourier transform
        ts_fourier = np.fft.rfft(ts[i])

        # Generate random phases
        random_phases = np.exp(1j * np.random.uniform(0, 2*np.pi, len(ts[i]) // 2 + 1))

        # Apply the random phases to the Fourier transform
        ts_fourier_randomized = ts_fourier * random_phases

        # Inverse Fourier transform
        ts_randomized[i] = np.fft.irfft(ts_fourier_randomized, len(ts[i]))
        
    return ts_randomized